## Exercise Class (May 2023)
### Exercises - Indicative Solutions - Discussion

In [ ]:
# Some auxiliary functions

# Reading a fast file
def readfasta(fastafile):
    import regex as re
    f=open(fastafile, 'r')
    seq = ""
    total = 0
    for line in f:
        x=re.match(">", line)
        if x == None:
            length=len(line)
            total=total+length
            seq=seq+line[0:length-1]
    f.close()
    seq=seq.replace('N','')
    return(seq)

# A maximum function
def maximum(x, y) :
    if x > y :
        return x
    else :
        return y


### 1. Palindromes. Finding them, extracting the greatest

In [ ]:
# Take 1: Iteration 

def longestpalindrome(s):
  palindromes=[]
  for i in range(1,len(s)):
    try:
        m=1 
        while s[i-m]==s[i+m]:
            palindromes.append(s[i-m:i+m+1])
            m+=1
    except:
        pass
  #return max(palindromes, key=len)
  return sorted(palindromes, key=len, reverse=True)

# Run
ecoliseq=readfasta("../files/ecoli.fa")
longestpalindrome(ecoliseq)


In [ ]:
# Take 2: Recursion

# we will build on a simple recursive function like the one given below

def palindrome(mystring):
    if len(mystring) < 2:
        return True
    if mystring[0] != mystring[-1]:
        return False
    return palindrome(mystring[1:-1])


# We will next scan the sequence of the genome and call the function 

def recursivePalindrome(s):
    palindromes = []
    for i in range(0,len(s)):
        k = 0
        subsequence = s[i]
        while (palindrome(subsequence)):
            palindromes.append(subsequence)
            k += 1
            subsequence = s[i-k:i+k]
        else:
            next            
            
    #return max(palindromes, key=len)
    return sorted(palindromes, key=len, reverse=True)

ecoliseq=readfasta("../files/ecoli.fa")
recursivePalindrome(ecoliseq)

#### Discussion

The solutions above do not yield the same palindrome. 
1. What is the problem with them?
2. Which one is correct?
3. Why is the one finding an odd-sized and the other an even-size palindrome?

### Slow Algorithm
In the above examples we implement a slower algorithm that is $O(nxm)$ where $n$ is the length of the longest sequence and $m$ is a factor that is related to the mean length of palindromes and their density inside the sequence (some sequences will take longer than others). 

### Odd/Even-Sized strings
Both approaches shown above (iterative/recursive) fail to report both odd/even-sized palindromes because of the nature of their implementation. A simple and elegant solution to this problem is to modify the input string by adding the same (neutral) character in-between every residue/initial character. This will make all strings appear as odd-sized. E.g. if the string 'GCATTATT' is converted to '#G#C#A#T#T#A#T#T#' will be an odd-lengthed '#A#T#T#A#'. Below the final solution for the iterative version. 

In [ ]:
def longestpalindrome(s):
    s=s.replace('','#')[1:-1]
    palindromes=[]
    for i in range(1,len(s)):
        try:
            m=1 
            while s[i-m]==s[i+m]:
                palindromes.append(s[i-m:i+m+1].replace('#',''))
                m+=1
        except:
            pass
    return sorted(palindromes, key=len, reverse=True)

longestpalindrome(ecoliseq)

### Longest Palindrome in linear time
Finding the longest palindrome is a problem that has been shown to be solved in $O(n)$ time by Manacher and later additions.   
  
Manacher uses a simple but intuitive technique to speed up the process by taking the context of the string into consideration. For example in the string 'abracarba', by the time the process reaches the central 'c', the algorithm knows that the four final strings 'arba' form part of a palindrome. It can then treat them differently by going to the end of the string and from there examine what happens *downstream* of the already identified palindrome. This concept of "context" is explored in many other problems of 
  
For those interested in this solution, you can find more [here](https://www.geeksforgeeks.org/manachers-algorithm-linear-time-longest-palindromic-substring-part-1/).

### 2. Non-mers in E.coli Genome

This was rather straight-forward as we had discussed it in class. Nevertheless, there is room for some discussion on optimization and the *assessment of statistical expectations*.   

Below I provide code taken from one of your assignments, slightly modified and with comments.

In [ ]:
with open ("../files/ecoli.fa") as l:
    l.readline()
    ecolseq =""
    for ln in l:
        lnseq = ln.strip()
        ecolseq += lnseq

#Find all the possible 10-mers that can be found with the 4 nucleotides
import itertools

k=10
bases = ["A","T","G","C"]
combos = itertools.product(bases, repeat=k)
strcomb = ("".join(y) for y in combos) #generators save memory compared to lists

#Find all k-mers/10-mers in the genome

kmers=[]
for i in range(len(ecolseq)-k+1):
    kmers.append(ecolseq[i:i+k])

#Isolation of all the unique ones (makes the procedure faster)
kmers = set(kmers)

#Calculating how many 10-mers are not included in the E.coli genome
count=0
nonkmers =[]
allk =0
res =0
for y in strcomb:
    allk += 1     #control to check if the theoritical 4^10 has been produced with the product functional tool 
    if y not in kmers:
        count +=1 #counting the 10-mers not included is same as len(nonkemers)/not necessary / check point
        nonkmers.append(y)
    else:         #not necessary / check point
        res+=1    #counting the included 10-mers / check point


import random

print("The number of non 10-mers is:",count)
print("Some of those non-mers are", random.sample(nonkmers,10))

# POINT: We can do the same without looping, simply by comparing sets
allkmers = set([''.join(p) for p in itertools.product(bases, repeat=k)])
nonmers = allkmers.difference(kmers)
print("Found:", len(nonmers), "nonmers in the genome of E.coli")

#Expected non-10-mers/ Theoritical check: 
theor = 4**10
obs = len(kmers)
exp = theor - obs
print("According to theory we would expect", exp,"non-mers since all the 10-mers are", theor,"and the observed in E.Coli are", obs)

# NOTE: The theoritical check is not in agreement with the ones found above with comparison. Not quite sure why.

# =================================================================================================================
# POINT: This is not correct from a theoretical viewpoint. In fact it is much more complex. Can you think of why?
# =================================================================================================================


#Another way to avoid for-loops and exhausting search is to immediately produce sets that contain unique 10-mers and calculate their difference:
#The reading of the file is common

# -> This is the point made above! (OK)

kmers2 = {ecolseq[i:i+k] for i in range(len(ecolseq)-k+1) }
combos = itertools.product(bases, repeat=k) 
#must to re-write the command since it's a generator that has been used above and needs to be generated to reused
strcomb2 = {"".join(y) for y in combos} #permuation with replacement we can make it a set as they all uniq
nonmers = strcomb2 - kmers2

print(len(nonmers))
print(random.sample(nonmers, 10)) #demonstartion of some non-mers

Question: Did anyone try to use the binary search? Would it be faster?

### 3. HGT and Clustering using odds Ratios

Again, the technical aspects of these were discussed in class. It really is a matter of careful and detailed implementation.

### 4. Gibbs Sampler

Due to lack of time I couldn't go through the details of all your code. Most of you were able to implement the Sampler but many could not make it converge to the actual result.  

Below I am attaching an indicative solution. We can return to it, to see what may go wrong and at which point in particular.

In [ ]:
# Gibbs Sampling to locate a motif in a set of sequences
# code
sequences = []
with open ('../files/motifs_in_sequence.fa') as file:
    for line in file:
        sequences.append(line.strip()) # 50 sequences as elements of a list. 100 bases each sequence
    

##Gibbs sampler##
import random
import numpy as np

def Gibbs_sampler(sequences,k): #k is the length of the motif, sequence is a list with the sequences
    
    dictionary = {'A':0,
                  'T':1,
                  'C':2,
                  'G':3}
    
    column_sum = len(sequences) #number of rows (50) or number of sequences
    length = len(sequences[0]) #number of columns or number of nucleotides in seq
    Imax = 1.8*k #threshold of I
    
    pwm = np.zeros([4,k]) # A,T,C,G X len(motif)
    
    for seq in sequences:
        rand_start = random.randint(0, length-k) #pick a random nucleotide from each sequence
        motif = seq[rand_start:rand_start+k] #and take substring as the motif
       
        lst = enumerate(motif) #finding the index of each nucleotide in the motif to access the correct column
                               #and using the dictionary to access the correct row 
        for i in lst: #making the first random pwm
            pwm[dictionary[i[1]],i[0]]+=1
            
    pwm = pwm/column_sum
    
    information = np.zeros([1,k])
    count=0
    while (np.sum(information)) < Imax: #while information_content of the pwm 
        motives=[]                      #is less than the threshold
        
        information_old = np.sum(information) #keeping the previous value of information contect
                                             #to check convergence in case the theshold 
        for row in range (column_sum):                #is never reached
            maxx=0
            rand_seq = random.randint(0, column_sum-1) #pick a random index - sequence
            seq = sequences[rand_seq]               
            for i in range(len(seq)-k):   #take each k-mer from the sequence 
                score = 0
                substring = seq[i:i+k]
                lst = enumerate(substring)
                
                for j in lst:                         #scoring each k-mer based on the pwm
                    score+=pwm[dictionary[j[1]],j[0]]   #keeping the motif with the highest score
                                                         #from each sequence
                if score > maxx:  
                    maxx = score
                    motif = substring
                    
            motives += [motif] #keep all the motifs with the highest score in the list motives
        
        pwm = np.zeros([4,k]) # A,T,C,G X len(motif) 
        
        for elem in motives: 
            lst = enumerate(elem)
            for i in lst:         #making the new pwm
                pwm[dictionary[i[1]],i[0]]+=1
                 
        pwm = pwm/column_sum
        
        information = np.zeros([1,k]) #computing the information of each position
        for i in range(k):
            information[0,i] = 2-abs(sum([elem*np.log2(elem) for elem in pwm[:,i] if elem > 0]))
            

        if abs(information_old - np.sum(information)) <= 0.5: #ckecking convergence 
            count+=1
            if count == 10: #if the difference of the information content is less or equal to 0.5
                break       #for consecutive 10 iterations then break
        else:
            count=0
    
    max_index_col = np.argmax(pwm, axis=0) #extracting the motif according to the   
                                           #highest frequency of each nucleotide in each position
    motif=''
    for values in max_index_col:
        for keys in dictionary.keys():
            if values == dictionary[keys]:
                motif+= keys
        
    return pwm,information,motif


#repeat the algorithmm 100 times for each k (3 to 7) and keep the pwm and motif with the highest infromation_content
#this process takes approximately 4min (in my computer)
    
####100-cycled GIbbs#####
for k in range (3,8):
    highest_info = 0
    for i in range (100):
        summ=0
        pwm, information_content,motif = Gibbs_sampler(sequences,k)
        summ+=np.sum(information_content)
        if summ > highest_info:
            highest_info = summ
            pwm_ret = pwm
            motif_ret = motif
        
    print('\nThe information content of the motif divided by it\'s length is:',highest_info/k) #divide by length to normalize and compare among other k
    print('The pwm of the motif is:\n',pwm_ret)
    print('The motif is:',motif_ret)



#To find the motifs for each k with the highest information content i have to repeat gibbs sampler many times because of the randomness that takes place
#The motif that returns the highest scaled information content is GAT (k = 3, I/k = 1.9528 or 2!) but i think that the motif that we are looking for
#is the motif GATA (k=4, I/k = 1.857) which contains GAT and is contained in all the longer found motifs.
#(Also we know the existence of the GATA transcription factors and indeed GATA is part of these binding sites)
#Sometimes the algorithm returns other 3-mers (eg ATG,GGC,AAG) with high information content as well but it
#is noted that GATA is returned almost in every repetition of the 100-cycled Gibbs which means that it may be the only 4-mer motif with so high information content.
#Finally i have to report that the threshold that i chose for checking convergence may not be the best choice and should be stricter
#Either way i kept this threshold because otherwise i would have made the algorithm a lot slower (is already slow though :P).

### Longest Common Subsequence between two sequences

In [3]:
### Longest Common Subsequence
def longest_common_contiguous_subsequence(seq1, seq2):
    m, n = len(seq1), len(seq2)
    
    # Create a 2D array to store lengths of longest common contiguous subsequence
    # Initialize all values to 0
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    
    # Variable to keep track of the length of the longest common contiguous subsequence
    max_length = 0
    # Variable to store the ending index of the longest common contiguous subsequence in seq1
    end_index_seq1 = 0

    # Build the dp array from bottom up
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if seq1[i - 1] == seq2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1] + 1
                if dp[i][j] > max_length:
                    max_length = dp[i][j]
                    end_index_seq1 = i
            else:
                dp[i][j] = 0

    # The longest common contiguous subsequence
    longest_common_substr = seq1[end_index_seq1 - max_length:end_index_seq1]
    
    return longest_common_substr

# Example usage
seq1 = "AGGTAB"
seq2 = "XXAGTAYBYBYB"
print(f"The Longest Common Contiguous Subsequence is: {longest_common_contiguous_subsequence(seq1, seq2)}")


The Longest Common Contiguous Subsequence is: GTA


### Creation of Suffix Array

In [5]:
def build_suffix_array(seq):
    # Generate all suffixes of the given sequence along with their starting indices
    suffixes = [(seq[i:], i) for i in range(len(seq))]
    
    # Sort the suffixes lexicographically
    suffixes.sort()
    
    # Extract and return the starting indices of the sorted suffixes
    suffix_array = [suffix[1] for suffix in suffixes]
    
    return suffix_array

# Example usage
seq = "AGCCTAGACACAGTACCACGTAB"
suffix_array = build_suffix_array(seq)
print(f"Suffix Array: {suffix_array}")

Suffix Array: [21, 7, 9, 14, 17, 5, 0, 11, 22, 8, 16, 10, 15, 2, 18, 3, 6, 1, 19, 12, 20, 13, 4]


### BWT and Compression with RLE

In [10]:
def build_suffix_array(seq):
    suffixes = [(seq[i:], i) for i in range(len(seq))]
    suffixes.sort()
    suffix_array = [suffix[1] for suffix in suffixes]
    return suffix_array

def reorder_sequence(seq, suffix_array):
    reordered_seq = ''.join(seq[i] for i in suffix_array)
    return reordered_seq

def run_length_encoding(seq):
    if not seq:
        return ""
    
    encoding = []
    prev_char = seq[0]
    count = 1
    
    for char in seq[1:]:
        if char == prev_char:
            count += 1
        else:
            encoding.append(prev_char + str(count))
            prev_char = char
            count = 1
    encoding.append(prev_char + str(count))
    
    return ''.join(encoding)

# Example usage
seq = "AGGTABAGGTAB$"
suffix_array = build_suffix_array(seq)
reordered_seq = reorder_sequence(seq, suffix_array)
compressed_seq = run_length_encoding(reordered_seq)

print(f"Original Sequence: {seq}")
print(f"Suffix Array: {suffix_array}")
print(f"Reordered Sequence: {reordered_seq}")
print(f"Compressed Sequence: {compressed_seq}")

Original Sequence: AGGTABAGGTAB$
Suffix Array: [12, 10, 4, 6, 0, 11, 5, 7, 1, 8, 2, 9, 3]
Reordered Sequence: $AAAABBGGGGTT
Compressed Sequence: $1A4B2G4T2


### Longest Palindrome of a Sequence


In [11]:
### Take 1. Simple Expansion Around Center
def longest_palindromic_substring_expand_center(s):
    n = len(s)
    if n == 0:
        return ""
    
    start = 0
    end = 0
    
    for i in range(n):
        len1 = expand_around_center(s, i, i)
        len2 = expand_around_center(s, i, i + 1)
        max_len = max(len1, len2)
        
        if max_len > end - start:
            start = i - (max_len - 1) // 2
            end = i + max_len // 2
    
    return s[start:end + 1]

def expand_around_center(s, left, right):
    while left >= 0 and right < len(s) and s[left] == s[right]:
        left -= 1
        right += 1
    return right - left - 1

# Example usage
dna_sequence = "AGGTABAGGTAB"
longest_palindrome = longest_palindromic_substring_expand_center(dna_sequence)
print(f"Longest Palindromic Substring (Expand Around Center): {longest_palindrome}")

Longest Palindromic Substring (Expand Around Center): ABA


In [12]:
### Take 2. Using Dynamic Programming
def longest_palindromic_substring_dp(s):
    n = len(s)
    if n == 0:
        return ""
    
    # Initialize a table to store palindrome status
    dp = [[False] * n for _ in range(n)]
    
    start = 0
    max_length = 1
    
    # All substrings of length 1 are palindromes
    for i in range(n):
        dp[i][i] = True
    
    # Check for substrings of length 2
    for i in range(n - 1):
        if s[i] == s[i + 1]:
            dp[i][i + 1] = True
            start = i
            max_length = 2
    
    # Check for lengths greater than 2
    for length in range(3, n + 1):
        for i in range(n - length + 1):
            j = i + length - 1
            
            if s[i] == s[j] and dp[i + 1][j - 1]:
                dp[i][j] = True
                start = i
                max_length = length
    
    return s[start:start + max_length]

# Example usage
dna_sequence = "AGGTGABAGGTAB"
longest_palindrome = longest_palindromic_substring_dp(dna_sequence)
print(f"Longest Palindromic Substring (DP): {longest_palindrome}")

Longest Palindromic Substring (DP): GABAG
